In [1]:
import scipy.stats as ss
import numpy as np
import random
from math import exp, sqrt
np.set_printoptions(precision=4)
import time

In [2]:
def calculate_S_T(S, sigma, r, T):
    """
    estimate epsilon，compute S_T
    using generalized wiener process
    """
    
    return S * exp((r - 0.5 * sigma ** 2) * T + sigma * sqrt(T) * random.gauss(0.0, 1.0))

def getRoundStockPrice(S, sigma, r, N, T, rounds, paths):
    """
    This function is used to simulate N random path for a stock
    S: Stock price
    sigma: volatility rate
    r: interest rate
    N: N steps
    T: time interval
    paths: how many paths for stock price
    """
    firstRound = [S] * paths    #creating the first row 
    nextRound = firstRound.copy() #copying the first row
    for i in range(rounds):
        nextRound1 = nextRound[-paths:]        
        for s in nextRound1:
            nextS = calculate_S_T(s, sigma, r, T)
            nextRound.append(nextS)
    nextRound = np.array(nextRound).reshape((-1, paths)).T
    return nextRound

S = 100
sigma = 0.2
r = 0.1
N = 5
T = 1/N

paths = 10
S_Matrix = getRoundStockPrice(S, sigma, r, N, T, N, paths)
S_Matrix

array([[100.    ,  95.227 ,  97.1684,  93.9107,  87.7259,  89.1876],
       [100.    , 103.9663, 108.17  , 118.4833, 141.1923, 160.7761],
       [100.    ,  91.8015,  92.7929, 111.0398, 104.4469, 111.9558],
       [100.    ,  89.3186,  89.5722,  84.0282,  95.0404,  95.5193],
       [100.    , 119.5032, 158.9755, 150.6048, 132.2399, 118.4127],
       [100.    ,  97.36  ,  99.8569, 105.7206,  93.3808, 102.2375],
       [100.    , 103.0864, 107.5693,  97.0691, 100.591 ,  96.8766],
       [100.    , 111.0808, 115.1753, 123.919 , 121.9681, 137.5874],
       [100.    ,  97.705 , 113.5782, 115.864 , 114.3571, 136.96  ],
       [100.    ,  97.1221, 102.5056, 119.5147, 108.7946, 116.3197]])

In [3]:
calculate_S_T(S, sigma, r, T)

88.94340503083669

In [2]:
def LSM(S0, K, T, r, sigma, option_type, N=100, paths=100):
    """
    This program is used to price american option using lsm monte carlo method
    S0:Stock price
    K:strike price
    r:interest rate
    sigma: volatility rate
    option_type:"call" or "put" American option
    N:number of steps
    paths: number of paths
    """
    start_time = time.time()
    dt = T/(N-1)        # time interval
    df = np.exp(-r * dt)  # discount factor per time time interval

    X0 = np.zeros((paths,1)) #creating an array with paths rows and one column
    #creating random path using "scipy.stats" package which return a normal continuous random variable
    #The location (loc) keyword specifies the mean. The scale (scale) keyword specifies the standard deviation.
    increments = ss.norm.rvs(loc=(r - sigma**2/2)*dt, scale=np.sqrt(dt)*sigma, size=(paths,N-1))
    X = np.concatenate((X0,increments), axis=1).cumsum(1)#Join a sequence of arrays along an existing axis.
    S = S0 * np.exp(X) #stock price in each step
    if option_type == "put":
        H = np.maximum(K - S, 0)   # intrinsic values for put option,H=ev
    if option_type == "call":
        H = np.maximum(S - K, 0)   # intrinsic values for call option
    V = np.zeros_like(H)           # value matrix
    V[:,-1] = H[:,-1]              # let the last column of V be same as H

    # Valuation by LS Method
    for t in range(N-2, 0, -1):
        #when >0 return True, otherwise,False
        good_paths = H[:,t] > 0    
        # polynomial regression：selecting parts with EV > 0 
        rg = np.polyfit( S[good_paths, t], V[good_paths, t+1] * df, 2)    
        # estimate E(HV)
        C = np.polyval( rg, S[good_paths,t] )                             
        # if E(HV)<EV，exercise
        exercise = np.zeros( len(good_paths), dtype=bool)
        exercise[good_paths] = H[good_paths,t] > C
        V[exercise,t] = H[exercise,t]
        V[exercise,t+1:] = 0
        discount_path = (V[:,t] == 0)
        V[discount_path,t] = V[discount_path,t+1] * df
    V0 = np.mean(V[:,1]) * df  #return the final result 
    end_time = time.time()
    print('running time is '+ str(end_time - start_time)+'s')
    print(V0)

In [3]:
put = LSM(100, 100, 1, 0.1, 0.2, 'put', N=2000, paths=2000)

running time is 1.1863017082214355s
4.754518839028211
